# **LAB 3 - PART 1 SVM Kernel Hard Prediction**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

#### **Read csv file**

In [ ]:
### START CODE HERE ###
df = pd.read_csv("/content/data_sample.csv")
df
### END CODE HERE ###

FileNotFoundError: [Errno 2] No such file or directory: '/content/data_sample.csv'

#### **Split the data for Train and Test**
First 5 rows for train and the rest for test.

In [ ]:
### START CODE HERE ###
train = df.head()
test = df.tail(len(df) - 5)

print(train)
print(test)
### END CODE HERE ###

Split targets and feature as y and x respectively.
- `y` : `Target`
- `x` : `Radius,Texture`

In [ ]:
### START CODE HERE ###
# Extract 'Target' column for training set
y_train = train.iloc[:, 0]

# Extract 'Radius' and 'Texture' columns for training set
x_train = train.iloc[:, 1:3]  # Assuming 'Radius' is at position 1 and 'Texture' is at position 2

# For test set, assuming 'test' is the DataFrame containing your test data
# Extract 'Target' column for test set
y_test = test.iloc[:, 0]

# Extract 'Radius' and 'Texture' columns for test set
x_test = test.iloc[:, 1:3]  # Assuming



### END CODE HERE ###

### **Create and observe the kernel behavior**

Here is the kernel function.
$$K_i(x_j,x_i) = exp(-\gamma||x_j-x_i||^2)$$
After obtaining the kernel values, you will perform a 3D visualization of $(x_1,x_2,K_i)$.

**Create the function that return all kernels as a list.**

In [ ]:
def matrix_calculate_kernel(x,x_train):
    kernels = []
    kernels_arr = []
    gamma = 1
    ### START CODE HERE ###


    n_samples = x_train.shape[0]
    kernel_value = []

    for i in range(x.shape[0]):
        for j in range(n_samples):
            xi = x.iloc[i,:]
            xj = x_train.iloc[j,:]
            distance_squared = np.sum((xi - xj) ** 2)
            kernel_value = np.exp(-gamma * distance_squared)
            kernels.append(kernel_value)
        kernels_arr.append(kernels)
        kernels = []
    return kernels_arr


    ### END CODE HERE ###


<details>

<summary>
<font size="3" color="orange">
<b>Expected output</b>
</font>
</summary>

It is not necessary to print the output like this. Simply compare the values obtained with the expected results.
- Kernel 1 :`[1.00000000e+00 1.68638147e-01 1.31720301e-06 1.65298888e-01 1.16785670e-02]`
- Kernel 2 :`[1.68638147e-01 1.00000000e+00 1.68752986e-09 1.20342323e-02 9.44523667e-06]`
- Kernel 3 :`[1.31720301e-06 1.68752986e-09 1.00000000e+00 2.85124368e-03 3.28082741e-07]`
- Kernel 4 :`[0.16529889 0.01203423 0.00285124 1.         0.00119453]`
- Kernel 5 :`[1.16785670e-02 9.44523667e-06 3.28082741e-07 1.19453296e-03 1.00000000e+00]`

</details>



In [ ]:
matrix_calculate_kernel(x_train,x_train)


Use the weight ($\theta_{1-n}$) and bias ($\theta_{0}$) base on your activity last.

In [ ]:
kernels_train = np.array(matrix_calculate_kernel(x_train,x_train))
print(kernels_train[0])
### START CODE HERE ###
w = np.array([0.5,-0.5,0.1,-0.5,0])
b = [0]

kernels_train[0]*w[0]
### END CODE HERE ###

Create the `compare_3dplot` function to plot difference raw kernels $(x_1,x_2,K_i)$ and weighted kernels $(x_1,x_2,\theta_i K_i)$.<br>
**NOTE** : Plot as subplots, and in each subplot, also plot $(x_1,x_2,0)$  with a different color to visualize the $(x_1,x_2)$ before applying the kernel.

<details>

<summary>
<font size="3" color="green">
<b>Hints</b>
</font>
</summary>

- There is a parameter : `projection='3d'`
</details>

<br>
<details>

<summary>
<font size="3" color="orange">
<b>Expected output</b>
</font>
</summary>



</details>


In [ ]:
# # %matplotlib widget
# def compare_3dplot(x_train,kernels,w):




#     ### START CODE HERE ###

#     # Create subplots
#     fig, axs = plt.subplots(1, len(kernels) + 1, figsize=(15, 5), subplot_kw={'projection': '3d'})

def compare_3dplot(x_train,kernels,w):

    # Create subplots
    fig, axs = plt.subplots(1, 2, figsize=(10, 5), subplot_kw={'projection': '3d'})

    # Plot raw kernels
    # axs[0].plot_trisurf(x_train['Radius'], x_train['Texture'], kernels)
    axs[0].scatter3D(x_train['Radius'], kernels, c='blue')
    axs[0].scatter3D(x_train['Texture'], kernels, c='green')
    axs[0].set_title('Raw Kernels')

    # Plot weighted kernels
    axs[1].set_title('Weighted Kernels')
    axs[1].scatter3D(x_train['Radius'], kernels*w, c='blue')
    axs[1].scatter3D(x_train['Texture'], kernels*w, c='green')
#     ### END CODE HERE ###

In [ ]:
compare_3dplot(x_train,kernels_train[0],w[0])

In [ ]:
compare_3dplot(x_train,kernels_train[1],w[1])

In [ ]:
compare_3dplot(x_train,kernels_train[2],w[2])

In [ ]:
compare_3dplot(x_train,kernels_train[3],w[3])

In [ ]:
compare_3dplot(x_train,kernels_train[4],w[4])

### **SVM decision function**
Create the SVM decision function that return the predicted class.
$$
K_i(x_j,x_i) = exp(-\gamma||x_j-x_i||^2) \\
h(X) = \sum_{i=1}^N\theta_i K_i + \theta_0\\

\hat{Y} =
\begin{cases}
    1 &  h \geq 0 \\
    0 & \text{otherwise}
\end{cases}

$$


In [ ]:
def svm_decision_function(x, x_train, w, b, gamma=1):
    pred_list = []
    kernels = np.array(matrix_calculate_kernel(x,x_train))
    # Take dot product between kernels and weights
    for i in range(len(x)) :
      decision = np.dot(kernels[i], w) + b

      # Make prediction
      if decision >= 0:
          pred_class = 1
      else:
          pred_class = 0
      pred_list.append(pred_class)

    return pred_list

y_pred_arr = svm_decision_function(x_test,x_train,w,b)

In [ ]:
x_train

### **Confusion matrix**
Create the confusion matrix from your prediction. Then, calculate the precision and recall values.<br><br>
<span style='color: red;'>**NOTE** : From scratch only.</span>


|            | Predicted Negative | Predicted Positive |
|------------|--------------------|--------------------|
| **Actual Negative** | True Negative (TN) | False Positive (FP) |
| **Actual Positive** | False Negative (FN) | True Positive (TP) |


$\text{Precision} = \frac{TP}{TP + FP}$

$\text{Recall} = \frac{TP}{TP + FN}$

<details>

<summary>
<font size="3" color="orange">
<b>Expected output</b>
</font>
</summary>

- The output should resemble this, but not be identical.</br>

```
Confusion Matrix:
[[1 0]
[0 2]]
Precision: 1.0
Recall: 1.0
```

</details>

In [ ]:
y_test_arr =  np.array(y_test.values)
y_pred_arr = np.array(y_pred_arr)
# y_pred_arr

In [ ]:
 ### START CODE HERE ###

TP = np.sum((y_pred_arr == 1) & (y_test_arr == 1))
TN = np.sum((y_pred_arr == 0) & (y_test_arr == 0))
FP = np.sum((y_pred_arr == 1) & (y_test_arr == 0))
FN = np.sum((y_pred_arr == 0) & (y_test_arr == 1))

cm = np.array([[TN, FP],
               [FN, TP]])

# Print confusion matrix
print("Confusion Matrix:")
print(cm)

# Calculate precision and recall
precision = TP / (TP + FP)
recall = TP / (TP + FN)

print(f"Precision: {precision:.3f}")
print(f"Recall: {recall:.3f}")




### END CODE HERE ###




### END CODE HERE ###

### Question
1. What would happen if we choose fewer support vectors (number of kernels)?
2. If weights are changed to w = [0.2, 0.4, 0.1, -0.2, 0.6], what impact would this have on the classification results, and what is the underlying reason for the changes?

#Ans
1.